# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096860


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:04<01:07,  2.41s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:10,  2.61s/it]

Rendering models:  13%|█▎        | 4/30 [00:08<00:51,  1.98s/it]

Rendering models:  17%|█▋        | 5/30 [00:08<00:36,  1.47s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:17<00:02,  1.84it/s]

Rendering models:  90%|█████████ | 27/30 [00:18<00:01,  1.97it/s]

Rendering models:  93%|█████████▎| 28/30 [00:18<00:01,  1.97it/s]

Rendering models:  97%|█████████▋| 29/30 [00:19<00:00,  1.83it/s]

not-logged-in-3b0b929f4475fd276561    0.000815
not-logged-in-673e227f920b24558cc1    0.002085
matiasd123                            0.001986
Jnursssmith5263                       0.002588
not-logged-in-f737655ff84da201cfb7    0.055387
Jonas_Cross                           0.004609
jnarayanbvg                           0.000948
Quarkgc                               0.001390
natehamer16                           0.001437
not-logged-in-e64bc0338d23b9608188    0.003167
not-logged-in-409241128adb6e9c5c86    0.046699
not-logged-in-9a45911eb71184fe7c7f    0.001786
JamesTDG                              0.000743
galaxypotato                          0.001475
acapirala                             0.000947
not-logged-in-3569dcff4dbd0ee90f50    0.001013
Lavadude                              0.015374
not-logged-in-4937d52fc3b062ae0d43    0.001780
clars915                              0.000799
MrMinecraft                           0.001294
fje17                                 0.001062
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())